In [ ]:
import numpy as np
from nltk.tokenize import TweetTokenizer


In [ ]:
with open('./emoji_emoticon_8_features_test_vector.txt', 'r') as f:
    emoticon_8_features_vectors = f.read().split("]")
    emoticon_8_features_result = []
    for i in range(len(emoticon_8_features_vectors)):
        emoticon_8_features_vectors[i] = emoticon_8_features_vectors[i][1:]
        each_emoticon_8_features_vector = emoticon_8_features_vectors[i].split()
        each_emoticon_8_features_vector = [float(item) for item in each_emoticon_8_features_vector]
        emoticon_8_features_result.append(each_emoticon_8_features_vector)

with open('./sentiment140_unigram_test_vector.txt', 'r') as f:
    sentiment140_unigram_vectors = f.read().split("]")
    sentiment140_unigram_result = []
    for i in range(len(sentiment140_unigram_vectors)):
        sentiment140_unigram_vectors[i] = sentiment140_unigram_vectors[i][1:]
        each_sentiment140_unigram_vector = sentiment140_unigram_vectors[i].split()
        each_sentiment140_unigram_vector = [float(item) for item in each_sentiment140_unigram_vector]
        sentiment140_unigram_result.append(each_sentiment140_unigram_vector)

dev_features = []
for i in range(len(sentiment140_unigram_result)):
    dev_features.append(emoticon_8_features_result[i] + sentiment140_unigram_result[i])

dev_features = [x for x in dev_features if x != []]

dev_features = np.array(dev_features)

In [ ]:
with open('./emoji_emoticon_8_features_train_vector.txt', 'r') as f:
    emoticon_8_features_vectors = f.read().split("]")
    emoticon_8_features_result = []
    for i in range(len(emoticon_8_features_vectors)):
        emoticon_8_features_vectors[i] = emoticon_8_features_vectors[i][1:]
        each_emoticon_8_features_vector = emoticon_8_features_vectors[i].split()
        each_emoticon_8_features_vector = [float(item) for item in each_emoticon_8_features_vector]
        emoticon_8_features_result.append(each_emoticon_8_features_vector)


with open('./sentiment140_unigram_train_vector.txt', 'r') as f:
    sentiment140_unigram_vectors = f.read().split("]")
    sentiment140_unigram_result = []
    for i in range(len(sentiment140_unigram_vectors)):
        sentiment140_unigram_vectors[i] = sentiment140_unigram_vectors[i][1:]
        each_sentiment140_unigram_vector = sentiment140_unigram_vectors[i].split()
        each_sentiment140_unigram_vector = [float(item) for item in each_sentiment140_unigram_vector]
        sentiment140_unigram_result.append(each_sentiment140_unigram_vector)

train_features = []
for i in range(len(sentiment140_unigram_result)):
    train_features.append(emoticon_8_features_result[i] + sentiment140_unigram_result[i])
train_features = [x for x in train_features if x != []]
train_features = np.array(train_features)

In [ ]:
import pandas as pd
import CMUTweetTagger
from collections import OrderedDict, defaultdict, Counter
import csv
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import emot

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tweet_tokenizer = TweetTokenizer()
    stopword_list=nltk.corpus.stopwords.words('english')
    tokens = tweet_tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def remove_url(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

def clean_df(file):
    data = pd.read_csv(file, sep='\t', names=["id", "polarity", "tweet"])
    data = data.drop_duplicates()
    data['tweet']=data['tweet'].apply(remove_stopwords)
    data['tweet']=data['tweet'].apply(remove_url)
    data["tweaet"] = data["tweet"].str.lower() # lowercase
    data = data.reset_index(drop=True)
    return data

train_data = clean_df("./data/dataset/2013_train_sarcasm_data.csv")
train_labels = train_data.polarity
result = []
for x in train_labels:
    if x == "positive":
        result.append(2)
    elif x == "negative":
        result.append(1)
    elif x == "neutral":
        result.append(0)
train_labels = np.array(result)

dev_data = clean_df("./data/dataset/2013_test_sarcasm_data.csv")
dev_labels = dev_data.polarity
result = []
for x in dev_labels:
    if x == "positive":
        result.append(2)
    elif x == "negative":
        result.append(1)
    elif x == "neutral":
        result.append(0)
dev_labels = np.array(result)

In [ ]:
print("dev labels: ", dev_labels) 
print("dev features:", dev_features) 
print("dev_labels shape: ", dev_labels.shape) 
print("dev_features shape:", dev_features.shape) 

In [ ]:
print("train_labels ", train_labels) 
print("train_features:", train_features) 
print("train_labels shape: ", train_labels.shape) 
print("train_features shape:", train_features.shape) 

In [ ]:

clf = SVC(kernel='linear', C=0.005, probability=True)

sample_weight = np.array([3.14 if i == 1 else 1 for i in train_labels])

clf.fit(train_features, train_labels, sample_weight = sample_weight)

predictions = clf.predict(dev_features)

print("A Classification Report showing the per-class Precision, Recall and F1-score \n\nC=0.005 \nNegative weight = 3.14 \n\n", metrics.classification_report(dev_labels, predictions,target_names=['neutral','negative','positive']))




clf = SVC(kernel='linear', C=1, probability=True)

sample_weight = np.array([3.14 if i == 1 else 1 for i in train_labels])

clf.fit(train_features, train_labels, sample_weight = sample_weight)

predictions = clf.predict(dev_features)

print("A Classification Report showing the per-class Precision, Recall and F1-score \n\nC=1 \nNegative weight = 3.14 \n\n", metrics.classification_report(dev_labels, predictions,target_names=['neutral','negative','positive']))


clf = SVC(kernel='linear', C=0.005, probability=True)

arr = []

for i in train_labels:
    if i == 1:
        arr.append(3.14)
    elif i == 2:
        arr.append(1.25)
    else:
        arr.append(1)


sample_weight = np.array(arr)
clf.fit(train_features, train_labels, sample_weight = sample_weight)

predictions = clf.predict(dev_features)

print("A Classification Report showing the per-class Precision, Recall and F1-score \n\nC=0.005 \nNegative weight = 3.14 \nPositive weight = 1.25 \n\n", metrics.classification_report(dev_labels, predictions,target_names=['neutral','negative','positive']))


clf = SVC(kernel='linear', C=1, probability=True)

arr = []

for i in train_labels:
    if i == 1:
        arr.append(3.14)
    elif i == 2:
        arr.append(1.25)
    else:
        arr.append(1)


sample_weight = np.array(arr)
clf.fit(train_features, train_labels, sample_weight = sample_weight)

predictions = clf.predict(dev_features)

print("A Classification Report showing the per-class Precision, Recall and F1-score \n\nC=1 \nNegative weight = 3.14 \nPositive weight = 1.25 \n\n", metrics.classification_report(dev_labels, predictions,target_names=['neutral','negative','positive']))



clf = SVC(kernel='linear', C=1, probability=True)

clf.fit(train_features, train_labels)

predictions = clf.predict(dev_features)

print("A Classification Report showing the per-class Precision, Recall and F1-score \n\nC=1 \nNo weight", metrics.classification_report(dev_labels, predictions,target_names=['neutral','negative','positive']))


clf = SVC(kernel='linear', C=0.005, probability=True)

clf.fit(train_features, train_labels)

predictions = clf.predict(dev_features)

print("A Classification Report showing the per-class Precision, Recall and F1-score \n\nC=0.005 \nNo weight", metrics.classification_report(dev_labels, predictions,target_names=['neutral','negative','positive']))


from sklearn.model_selection import GridSearchCV
  
param_grid = {'C': [0.005, 0.1, 0.5, 1], 
              'kernel': ['linear','rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
grid.fit(train_features, train_labels)

print(grid.best_params_)
  
print(grid.best_estimator_)

grid_predictions = grid.predict(dev_features)
  
print(classification_report(dev_labels, grid_predictions))